In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flo-data-20k/flo_data_20k.csv


**HW1 RFM Analytics on FLO dataset**

In [2]:
###############################################################
# RFM ile Müşteri Segmentasyonu (Customer Segmentation with RFM)
###############################################################

###############################################################
# İş Problemi (Business Problem)
###############################################################
# FLO müşterilerini segmentlere ayırıp bu segmentlere göre pazarlama stratejileri belirlemek istiyor.
# Buna yönelik olarak müşterilerin davranışları tanımlanacak ve bu davranış öbeklenmelerine göre gruplar oluşturulacak..

###############################################################
# Veri Seti Hikayesi
###############################################################

# Veri seti son alışverişlerini 2020 - 2021 yıllarında OmniChannel(hem online hem offline alışveriş yapan) olarak yapan müşterilerin geçmiş alışveriş davranışlarından
# elde edilen bilgilerden oluşmaktadır.

# master_id: Eşsiz müşteri numarası
# order_channel : Alışveriş yapılan platforma ait hangi kanalın kullanıldığı (Android, ios, Desktop, Mobile, Offline)
# last_order_channel : En son alışverişin yapıldığı kanal
# first_order_date : Müşterinin yaptığı ilk alışveriş tarihi
# last_order_date : Müşterinin yaptığı son alışveriş tarihi
# last_order_date_online : Muşterinin online platformda yaptığı son alışveriş tarihi
# last_order_date_offline : Muşterinin offline platformda yaptığı son alışveriş tarihi
# order_num_total_ever_online : Müşterinin online platformda yaptığı toplam alışveriş sayısı
# order_num_total_ever_offline : Müşterinin offline'da yaptığı toplam alışveriş sayısı
# customer_value_total_ever_offline : Müşterinin offline alışverişlerinde ödediği toplam ücret
# customer_value_total_ever_online : Müşterinin online alışverişlerinde ödediği toplam ücret
# interested_in_categories_12 : Müşterinin son 12 ayda alışveriş yaptığı kategorilerin listesi

###############################################################
# GÖREVLER
###############################################################

# GÖREV 1: Veriyi Anlama (Data Understanding) ve Hazırlama
           # 1. flo_data_20K.csv verisini okuyunuz.
           # 2. Veri setinde
                     # a. İlk 10 gözlem,
                     # b. Değişken isimleri,
                     # c. Betimsel istatistik,
                     # d. Boş değer,
                     # e. Değişken tipleri, incelemesi yapınız.
          

df = pd.read_csv("/kaggle/input/flo-data-20k/flo_data_20k.csv")
df_ = df.copy()

In [3]:
df.head()



,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0,1.0,139.99,799.38,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0,2.0,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0,2.0,189.97,395.35,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,81.98,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0,1.0,49.99,159.99,[AKTIFSPOR]


In [4]:
df.columns


Index(['master_id', 'order_channel', 'last_order_channel', 'first_order_date',
       'last_order_date', 'last_order_date_online', 'last_order_date_offline',
       'order_num_total_ever_online', 'order_num_total_ever_offline',
       'customer_value_total_ever_offline', 'customer_value_total_ever_online',
       'interested_in_categories_12'],
      dtype='object')

In [5]:
df.describe().T


,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.0,3.110855,4.225647,1.00,1.00,2.00,4.00,200.00
order_num_total_ever_offline,19945.0,1.913913,2.062880,1.00,1.00,1.00,2.00,109.00
customer_value_total_ever_offline,19945.0,253.922597,301.532853,10.00,99.99,179.98,319.97,18119.14
customer_value_total_ever_online,19945.0,497.321690,832.601886,12.99,149.98,286.46,578.44,45220.13


*aykırı değerler var*

In [6]:
df.isnull().sum()


master_id                            0
order_channel                        0
last_order_channel                   0
first_order_date                     0
last_order_date                      0
last_order_date_online               0
last_order_date_offline              0
order_num_total_ever_online          0
order_num_total_ever_offline         0
customer_value_total_ever_offline    0
customer_value_total_ever_online     0
interested_in_categories_12          0
dtype: int64

In [7]:
df.dtypes

master_id                             object
order_channel                         object
last_order_channel                    object
first_order_date                      object
last_order_date                       object
last_order_date_online                object
last_order_date_offline               object
order_num_total_ever_online          float64
order_num_total_ever_offline         float64
customer_value_total_ever_offline    float64
customer_value_total_ever_online     float64
interested_in_categories_12           object
dtype: object

In [8]:
df.groupby("master_id").agg({"master_id":"count"}) # her bir satır bir transaction değil , her bir satır müşterinin sipariş davranışı özeti

,master_id
master_id,
00016786-2f5a-11ea-bb80-000d3a38a36f,1
00034aaa-a838-11e9-a2fc-000d3a38a36f,1
000be838-85df-11ea-a90b-000d3a38a36f,1
000c1fe2-a8b7-11ea-8479-000d3a38a36f,1
000f5e3e-9dde-11ea-80cd-000d3a38a36f,1
...,...
fff1db94-afd9-11ea-b736-000d3a38a36f,1
fff4736a-60a4-11ea-8dd8-000d3a38a36f,1
fffacd34-ae14-11e9-a2fc-000d3a38a36f,1


In [9]:
 # 3. Omnichannel müşterilerin hem online'dan hemde offline platformlardan alışveriş yaptığını ifade etmektedir. Herbir müşterinin toplam
           # alışveriş sayısı ve harcaması için yeni değişkenler oluşturun.
           

            
df["total_order"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["total_purchase"] = df["customer_value_total_ever_online"] + df["customer_value_total_ever_offline"]
df.head()



,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,total_order,total_purchase
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0,1.0,139.99,799.38,[KADIN],5.0,939.37
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0,2.0,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.0,2013.55
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0,2.0,189.97,395.35,"[ERKEK, KADIN]",5.0,585.32
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,81.98,"[AKTIFCOCUK, COCUK]",2.0,121.97
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0,1.0,49.99,159.99,[AKTIFSPOR],2.0,209.98


In [10]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.0,3.110855,4.225647,1.00,1.00,2.00,4.00,200.00
order_num_total_ever_offline,19945.0,1.913913,2.062880,1.00,1.00,1.00,2.00,109.00
customer_value_total_ever_offline,19945.0,253.922597,301.532853,10.00,99.99,179.98,319.97,18119.14
customer_value_total_ever_online,19945.0,497.321690,832.601886,12.99,149.98,286.46,578.44,45220.13
total_order,19945.0,5.024768,4.742707,2.00,3.00,4.00,6.00,202.00
total_purchase,19945.0,751.244287,895.402173,44.98,339.98,545.27,897.78,45905.10


In [11]:
# 4. Değişken tiplerini inceleyiniz. Tarih ifade eden değişkenlerin tipini date'e çeviriniz.
for col in df.columns:
    if "date" in col:
        df[col] = pd.to_datetime(df[col])
df.dtypes

master_id                                    object
order_channel                                object
last_order_channel                           object
first_order_date                     datetime64[ns]
last_order_date                      datetime64[ns]
last_order_date_online               datetime64[ns]
last_order_date_offline              datetime64[ns]
order_num_total_ever_online                 float64
order_num_total_ever_offline                float64
customer_value_total_ever_offline           float64
customer_value_total_ever_online            float64
interested_in_categories_12                  object
total_order                                 float64
total_purchase                              float64
dtype: object

In [12]:
# 5. Alışveriş kanallarındaki müşteri sayısının, ortalama alınan ürün sayısının ve ortalama harcamaların dağılımına bakınız.
#df.pivot_table( value = "order_channel" , index = "master_id",columns= "" , aggfunc:"count")
df["order_channel"].value_counts()
df.groupby("order_channel").agg({"order_channel":"count" , "order_num_total_ever_online":"mean","customer_value_total_ever_online":"mean" , "order_num_total_ever_offline":"mean","customer_value_total_ever_offline":"mean" })
           
           

,order_channel,order_num_total_ever_online,customer_value_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline
order_channel,,,,,
Android App,9495,3.582412,573.870516,1.922486,249.622139
Desktop,2735,2.061792,317.898004,1.930896,270.884980
Ios App,2833,3.495941,625.591440,1.922697,266.042845
Mobile,4882,2.557968,374.524574,1.882630,245.750551


In [13]:
# 6. En fazla kazancı getiren ilk 10 müşteriyi sıralayınız.
df[["master_id","total_purchase"]].sort_values(by="total_purchase",ascending=False).head(10)

,master_id,total_purchase
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,45905.10
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,36818.29
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,33918.10
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,31227.41
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,20706.34
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,18443.57
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,16918.57
163,fef57ffa-aae6-11e9-a2fc-000d3a38a36f,12726.10
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,12282.24
18767,fc0ce7a4-9d87-11e9-9897-000d3a38a36f,12103.15


In [14]:
# 7. En fazla siparişi veren ilk 10 müşteriyi sıralayınız.
df[["master_id","total_order"]].sort_values(by="total_order",ascending=False).head(10)

,master_id,total_order
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,202.0
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,131.0
8783,a57f4302-b1a8-11e9-89fa-000d3a38a36f,111.0
2619,fdbe8304-a7ab-11e9-a2fc-000d3a38a36f,88.0
6322,329968c6-a0e2-11e9-a2fc-000d3a38a36f,83.0
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,82.0
9347,44d032ee-a0d4-11e9-a2fc-000d3a38a36f,77.0
10954,b27e241a-a901-11e9-a2fc-000d3a38a36f,75.0
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,70.0
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,70.0


In [15]:
# 8. Veri ön hazırlık sürecini fonksiyonlaştırınız.
def preProcess(df):
    df["total_order"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
    df["total_purchase"] = df["customer_value_total_ever_online"] + df["customer_value_total_ever_offline"]
    for col in df.columns:
        if "date" in col:
            df[col] = pd.to_datetime(df[col])



In [16]:
# GÖREV 2: RFM Metriklerinin Hesaplanması
from datetime import timedelta

analysis_date = df["last_order_date"].max() + timedelta(days=2)
rfm = df.groupby("master_id").agg({"last_order_date":lambda x: (analysis_date - x.max()).days,
                                  "total_order": lambda x : x,
                                   "total_purchase": lambda x : x 
                                  })


rfm.columns = ["Recency","Frequency","Monetary"]
rfm.head()


,Recency,Frequency,Monetary
master_id,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.0,776.07
00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.0,269.47
000be838-85df-11ea-a90b-000d3a38a36f,213,4.0,722.69
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.0,874.16
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.0,1620.33


In [17]:
# GÖREV 3: RF ve RFM Skorlarının Hesaplanması
rfm["Recency_score"] = pd.qcut(rfm["Recency"],q=5,labels = [5,4,3,2,1])
rfm["Frequency_score"] = pd.qcut(rfm["Frequency"].rank(method="first") , q=5 , labels = [1,2,3,4,5])
rfm["Monetary_score"] = pd.qcut(rfm["Monetary"],q=5, labels = [1,2,3,4,5])
rfm["RF_SCORE"] = rfm["Recency_score"].astype(str) + rfm["Frequency_score"].astype(str)
rfm["RFM_SCORE"] = rfm["RF_SCORE"] + rfm["Monetary_score"].astype(str)
rfm.sort_values(by=["Monetary"] , ascending=False).head(20)

,Recency,Frequency,Monetary,Recency_score,Frequency_score,Monetary_score,RF_SCORE,RFM_SCORE
master_id,,,,,,,,
5d1c466a-9cfd-11e9-9897-000d3a38a36f,32,202.0,45905.10,5,5,5,55,555
d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,98,68.0,36818.29,3,5,5,35,355
73fd19aa-9e37-11e9-9897-000d3a38a36f,14,82.0,33918.10,5,5,5,55,555
7137a5c0-7aad-11ea-8f20-000d3a38a36f,49,11.0,31227.41,4,5,5,45,455
47a642fe-975b-11eb-8c2a-000d3a38a36f,35,4.0,20706.34,4,3,5,43,435
a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,32,70.0,18443.57,5,5,5,55,555
d696c654-2633-11ea-8e1c-000d3a38a36f,49,70.0,16918.57,4,5,5,45,455
fef57ffa-aae6-11e9-a2fc-000d3a38a36f,20,37.0,12726.10,5,5,5,55,555
cba59206-9dd1-11e9-9897-000d3a38a36f,23,131.0,12282.24,5,5,5,55,555


In [18]:
rfm.sort_values(by=["Frequency"] , ascending=False).head(50)

,Recency,Frequency,Monetary,Recency_score,Frequency_score,Monetary_score,RF_SCORE,RFM_SCORE
master_id,,,,,,,,
5d1c466a-9cfd-11e9-9897-000d3a38a36f,32,202.0,45905.10,5,5,5,55,555
cba59206-9dd1-11e9-9897-000d3a38a36f,23,131.0,12282.24,5,5,5,55,555
a57f4302-b1a8-11e9-89fa-000d3a38a36f,209,111.0,10383.44,2,5,5,25,255
fdbe8304-a7ab-11e9-a2fc-000d3a38a36f,336,88.0,8572.23,1,5,5,15,155
329968c6-a0e2-11e9-a2fc-000d3a38a36f,57,83.0,4240.36,4,5,5,45,455
73fd19aa-9e37-11e9-9897-000d3a38a36f,14,82.0,33918.10,5,5,5,55,555
44d032ee-a0d4-11e9-a2fc-000d3a38a36f,110,77.0,5184.05,3,5,5,35,355
b27e241a-a901-11e9-a2fc-000d3a38a36f,61,75.0,5297.88,4,5,5,45,455
d696c654-2633-11ea-8e1c-000d3a38a36f,49,70.0,16918.57,4,5,5,45,455


In [19]:
rfm.sort_values(by=["Recency"] , ascending=False).head(50)

,Recency,Frequency,Monetary,Recency_score,Frequency_score,Monetary_score,RF_SCORE,RFM_SCORE
master_id,,,,,,,,
6c3295f0-ac8e-11e9-a2fc-000d3a38a36f,367,3.0,491.58,1,2,3,12,123
473361e2-5b3b-11ea-b969-000d3a38a36f,367,2.0,239.48,1,1,1,11,111
ab6b731a-1da2-11ea-8bf2-000d3a38a36f,367,3.0,509.94,1,2,3,12,123
a4e242ba-22e5-11ea-8523-000d3a38a36f,367,3.0,829.20,1,2,4,12,124
f4595f52-f11e-11e9-9346-000d3a38a36f,367,3.0,514.76,1,3,3,13,133
f09f330c-a851-11e9-a2fc-000d3a38a36f,367,6.0,452.93,1,4,2,14,142
9a8f9dc0-ad34-11e9-a2fc-000d3a38a36f,367,5.0,1029.90,1,4,5,14,145
4b3c7d62-a90e-11e9-a2fc-000d3a38a36f,367,4.0,465.45,1,3,3,13,133
88f50d9e-543b-11ea-b1db-000d3a38a36f,367,2.0,289.98,1,1,1,11,111


In [20]:
# GÖREV 4: RF Skorlarının Segment Olarak Tanımlanması
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

rfm["SEGMENT"] = rfm['RF_SCORE'].replace(seg_map, regex=True)

rfm.head(50)


,Recency,Frequency,Monetary,Recency_score,Frequency_score,Monetary_score,RF_SCORE,RFM_SCORE,SEGMENT
master_id,,,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.0,776.07,5,4,4,54,544,champions
00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.0,269.47,1,2,1,12,121,hibernating
000be838-85df-11ea-a90b-000d3a38a36f,213,4.0,722.69,2,3,4,23,234,at_Risk
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.0,874.16,5,4,4,54,544,champions
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.0,1620.33,5,4,5,54,545,champions
00136ce2-a562-11e9-a2fc-000d3a38a36f,203,2.0,359.45,2,1,2,21,212,hibernating
00142f9a-7af6-11eb-8460-000d3a38a36f,25,3.0,404.94,5,2,2,52,522,potential_loyalists
0014778a-5b11-11ea-9a2c-000d3a38a36f,26,3.0,727.43,5,2,4,52,524,potential_loyalists
0018c6aa-ab6c-11e9-a2fc-000d3a38a36f,126,2.0,317.91,3,1,2,31,312,about_to_sleep


In [21]:
# GÖREV 5: Aksiyon zamanı!
           # 1. Segmentlerin recency, frequnecy ve monetary ortalamalarını inceleyiniz.
rfm[["Recency","Frequency","Monetary","SEGMENT"]].groupby("SEGMENT").agg("mean").sort_values(by="Monetary" , ascending=False)

,Recency,Frequency,Monetary
SEGMENT,,,
cant_loose,235.159129,10.716918,1481.652446
champions,17.142187,8.965104,1410.708938
loyal_customers,82.557926,8.356444,1216.257224
at_Risk,242.328997,4.470178,648.325038
need_attention,113.037221,3.739454,553.436638
potential_loyalists,36.869744,3.310769,533.741344
hibernating,247.426303,2.391474,362.583299
about_to_sleep,114.031649,2.406573,361.649373
new_customers,17.976226,2.000000,344.049495


In [22]:

# 2. RFM analizi yardımı ile 2 case için ilgili profildeki müşterileri bulun ve müşteri id'lerini csv ye kaydediniz.
       # a. FLO bünyesine yeni bir kadın ayakkabı markası dahil ediyor. Dahil ettiği markanın ürün fiyatları genel müşteri tercihlerinin üstünde. Bu nedenle markanın
       # tanıtımı ve ürün satışları için ilgilenecek profildeki müşterilerle özel olarak iletişime geçeilmek isteniliyor. Sadık müşterilerinden(champions,loyal_customers),
       # ortalama 250 TL üzeri ve kadın kategorisinden alışveriş yapan kişiler özel olarak iletişim kuralacak müşteriler. Bu müşterilerin id numaralarını csv dosyasına
       # yeni_marka_hedef_müşteri_id.cvs olarak kaydediniz.
       # b. Erkek ve Çoçuk ürünlerinde %40'a yakın indirim planlanmaktadır. Bu indirimle ilgili kategorilerle ilgilenen geçmişte iyi müşteri olan ama uzun süredir
       # alışveriş yapmayan kaybedilmemesi gereken müşteriler, uykuda olanlar ve yeni gelen müşteriler özel olarak hedef alınmak isteniliyor. Uygun profildeki müşterilerin id'lerini csv dosyasına indirim_hedef_müşteri_ids.csv
       # olarak kaydediniz.

#a

rfm[(rfm["SEGMENT"] == "champions") | (rfm["SEGMENT"] == "loyal_customers")].to_csv("yeni_marka_hedef_müşteri_id.csv")
rfm[(rfm["SEGMENT"] == "champions") | (rfm["SEGMENT"] == "loyal_customers")].head()






,Recency,Frequency,Monetary,Recency_score,Frequency_score,Monetary_score,RF_SCORE,RFM_SCORE,SEGMENT
master_id,,,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.0,776.07,5,4,4,54,544,champions
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.0,874.16,5,4,4,54,544,champions
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.0,1620.33,5,4,5,54,545,champions
003903e0-abce-11e9-a2fc-000d3a38a36f,68,5.0,577.47,4,4,3,44,443,loyal_customers
004bec18-9f7a-11e9-a2fc-000d3a38a36f,99,12.0,684.91,3,5,4,35,354,loyal_customers


In [23]:
#b
#rfm[(rfm["SEGMENT"] == "about_to_sleep" ) | (rfm["SEGMENT"] =="new_customers") ]
discount_group = rfm[(rfm["SEGMENT"] == "about_to_sleep" ) | (rfm["SEGMENT"] =="new_customers") ].merge(df[["master_id","interested_in_categories_12"]],on="master_id")
discount_group[discount_group["interested_in_categories_12"].str.lower().str.contains("erkek|cocuk")].to_csv("indirim_hedef_müşteri_ids.csv")
discount_group[discount_group["interested_in_categories_12"].str.lower().str.contains("erkek|cocuk")].head()

,master_id,Recency,Frequency,Monetary,Recency_score,Frequency_score,Monetary_score,RF_SCORE,RFM_SCORE,SEGMENT,interested_in_categories_12
0,0018c6aa-ab6c-11e9-a2fc-000d3a38a36f,126,2.0,317.91,3,1,2,31,312,about_to_sleep,[AKTIFCOCUK]
2,00414e84-adc6-11e9-a2fc-000d3a38a36f,120,3.0,303.44,3,2,2,32,322,about_to_sleep,[ERKEK]
5,00c2168c-a875-11e9-a2fc-000d3a38a36f,120,3.0,381.96,3,2,2,32,322,about_to_sleep,[ERKEK]
12,0179158a-aac8-11e9-a2fc-000d3a38a36f,8,2.0,110.87,5,1,1,51,511,new_customers,"[ERKEK, KADIN]"
14,017d86e2-5599-11eb-9e65-000d3a38a36f,134,2.0,154.97,3,1,1,31,311,about_to_sleep,"[AKTIFCOCUK, KADIN]"


In [24]:
# GÖREV 6: Tüm süreci fonksiyonlaştırınız.
def rfm_process(df):
    # RFM Metriklerinin Hesaplanması
    analysis_date = df["last_order_date"].max() + timedelta(days=2)
    rfm = df.groupby("master_id").agg({"last_order_date":lambda x: (analysis_date - x.max()).days,
                                      "total_order": lambda x : x,
                                       "total_purchase": lambda x : x 
                                      })


    rfm.columns = ["Recency","Frequency","Monetary"]
    #RF ve RFM Skorlarının Hesaplanması
    rfm["Recency_score"] = pd.qcut(rfm["Recency"],q=5,labels = [5,4,3,2,1])
    rfm["Frequency_score"] = pd.qcut(rfm["Frequency"].rank(method="first") , q=5 , labels = [1,2,3,4,5])
    rfm["Monetary_score"] = pd.qcut(rfm["Monetary"],q=5, labels = [1,2,3,4,5])
    rfm["RF_SCORE"] = rfm["Recency_score"].astype(str) + rfm["Frequency_score"].astype(str)
    rfm["RFM_SCORE"] = rfm["RF_SCORE"] + rfm["Monetary_score"].astype(str)
    
    seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_Risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions'
    }

    rfm["SEGMENT"] = rfm['RF_SCORE'].replace(seg_map, regex=True)
    
    
    
    